FR - Le présent notebook montre comment se connecter à un serveur MinIO qui expose un service S3 et comment téléverser un fichier Zarr dans un contenant (bucket) <br>
EN - This notebook shows how to connect to a MinIO server that exposes an S3 service and upload a Zarr file to a bucket

In [ ]:
# We need to import urllib3 to disable certificate verification when MinIO servers were not set up with CA-level certificates (e.g. self signed certificates)
import urllib3
import pathlib
import getpass # getpass allows to pass credentials and other sensitive information securely
from minio.api import Minio # Even though doc says "from minio import Minio" this will not work with MinIO 7.1.16 (current as of 09/2023)

In [ ]:
# In web console terminal : kubectl get secret noobaa-admin -n openshift-storage -o json | jq -r '.data.AWS_ACCESS_KEY_ID|@base64d'
access_key = getpass.getpass(prompt='MinIO access key: ')

In [ ]:
# In web console terminal : kubectl get secret noobaa-admin -n openshift-storage -o json | jq -r '.data.AWS_SECRET_ACCESS_KEY|@base64d'
secret_access_key = getpass.getpass(prompt='MinIO secret access key: ')

In [ ]:
endpoint = getpass.getpass(prompt='MinIO endpoint without http.s and with port speficied after colon: ')

In [ ]:
client = Minio(endpoint,
    access_key=access_key,
    secret_key=secret_access_key,
    secure=True, # needed to ensure https; avoids "http not allowed on https" errors
    http_client=urllib3.PoolManager(cert_reqs='CERT_NONE')
    ) # explicitly not care about certificate; not for a production server!

In [ ]:
bucket_list = client.list_buckets()

In [ ]:
bucket_list

In [ ]:
zarr_file = "hrdps-2023091112-TT.zarr" # this is an example; `zarr_file` needs to be accessible locally

In [ ]:
bucket_name = getpass.getpass(prompt='Bucket name ')

In [ ]:
# Given `client`, `bucket_name` defined above and
# a Zarr file (a directory that has the name of the dataset and contains the series of files and sub-directories that make up the zarr "file")

def upload_zarr_directory(client, bucket_name, local_directory):
    try:
        # Validate the arguments
        assert isinstance(client, Minio), "client must be an instance of Minio"
        assert isinstance(bucket_name, str), "bucket_name must be a string"
        assert isinstance(local_directory, str), "local_directory must be a string"

        # Check if the bucket exists
        if not client.bucket_exists(bucket_name):
            raise ValueError("Bucket '{}' does not exist on the client".format(bucket_name))
        
        # Check if the zarr_file exists

        if not pathlib.Path(zarr_file).is_file():
            raise ValueError(f"{zarr_file} is not a valid local file.")

        for file_path in pathlib.Path(local_directory).glob('**/*'):
            if file_path.is_file():
                object_name = str(pathlib.Path(zarr_filename) / file_path.relative_to(local_directory))
                client.fput_object(bucket_name, object_name, str(file_path))

    except (AssertionError, ValueError) as e:
        raise ValueError(str(e))

In [ ]:
# Use function above to upload the whole bit
upload_zarr_directory(client, bucket_name, zarr_file)